In [2]:
from get_data import get_data
import pandas as pd
from random import sample
import re
import os

X, y = get_data()


X = pd.DataFrame(X, columns = ["reviewText"])
y = pd.Series(y)


y = y.where(X.reviewText.str.contains(r'[a-zA-Z]'))
X = X.where(X.reviewText.str.contains(r'[a-zA-Z]'))
X = X.reviewText.str.replace('\n', ' ')



ranges = [100, 500, 2000]

neg = X[y == 0]
pos = X[y == 1]

for r in ranges:
    print('\nProducing subset for size:', r)
    r = r*10
    text_neg = sample(list(neg), r//2)
    text_neg = ["0"+x for x in text_neg]
    text_pos = sample(list(pos), r//2)
    text_pos = ["1"+x for x in text_pos]
    text = text_neg+text_pos
    
    pattern1 = re.compile(r'([^0-9a-zA-Z\s])\1+(?=[a-z0-9A-Z])')
    pattern2 = re.compile(r'([^0-9a-zA-Z\s])\1+')
    pattern3 = re.compile(r'<[^>]>')
    for i, t in enumerate(text):
        t = re.sub(pattern1, r'\1 ', t)
        t = re.sub(pattern2, r'\1', t)
        t = re.sub(pattern3, r'', t)
        text[i] = t


    # fix "...blabl -> ... blabl" and reduce "... -> ."
    s_before = len(text)
    text[:] = filter(lambda sen: len(sen) <= 35, text)
    print('Found short sentences:',len(text))
    
    r = r//10
    df = get_data("n_{}".format(r))
    c = Counter(df["sentiment"])
    missing_neg = r//2 - c["0"]
    missing_pos = r//2 - c["1"]
    #"incase one is negative"
    if missing_neg < 0:
        missing_pos -= abs(missing_neg)
        missing_neg = 0
    elif missing_pos < 0:
        missing_neg -= abs(missing_pos)
        missing_pos = 0
    cwd = os.getcwd().replace("\\", "/") 
    with open(f'../Data/subsets/n_{r}.txt', 'a') as f:
        for i in text:
            label = i[0]
            hue = i[1:]
            if label == "0" and missing_neg > 0:
                f.write(f'{label}\t{hue}\n')
                missing_neg -= 1
            if label == "1" and missing_pos > 0:
                f.write(f'{label}\t{hue}\n')
                missing_pos -= 1

    


Producing subset for size: 100
Found short sentences: 201

Producing subset for size: 500
Found short sentences: 951

Producing subset for size: 2000
Found short sentences: 3730


In [3]:
from get_data import get_data
from collections import Counter
ranges = 500
df = get_data(type = "n_500")
c = Counter (df["sentiment"])
missing_neg = ranges//2 - c["0"]
missing_pos = ranges//2 - c["1"]

print(missing_neg, missing_pos)
    
        


0 0


In [18]:
X, Y = get_data()

In [21]:
import pandas as pd

In [28]:
X = pd.DataFrame(X, columns = ["reviewText"])

In [50]:
cwd = os.getcwd().replace("\\", "/")
f'{cwd}/Data/subsets/n_{r}_for_gpt.txt'

'C:/Users/Vikto/Desktop/meh/NLPPhard/modules/Data/subsets/n_100_for_gpt.txt'

In [9]:
df = get_data("n_100")

In [1]:
from get_data import get_data
get_data(type = "gpt_10")

(['Sigh. Conor, what happened?  For the past decade you\'ve given us the most amazing lyrics, the most stripped down (or fancifully gussied up) tunes and then you go and drop this mess on us?  Your albums prior to this have all been brilliant in their own unique way but "The People\'s Key" does nothing to advance the dialogue...stilted lyrics, out of place electro effects and a rambling, incoherent vein throughout make this one to pass on and pretend it never happened.  A little more restraint in the studio would be greatly appreciated next time around, gentlemen.  Signed, Someone who gave all your other albums 5 stars and would have given "Lifted" 6 stars if that were possible.  Your well earned 1 star ranking reflects both the quality of the music as well as the overwhelming let-down factor.',
  "man zapp was funky from the start discovered by bootsy collins and promoted by george clinton these boys couldn't help but be funky and set the world on fire cause more bounce had everybody 

In [1]:
from clean_data import clean_eda

In [3]:
clean_eda()